<a href="https://colab.research.google.com/github/MeharJulka/APUSH-Chatbot/blob/APUSH-Chatbox/Mehar_Julka_Final_Code_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  langchain_community  langchain_huggingface  huggingface_hub transformers colorama torch pdfminer.six chromadb langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s et

In [ ]:
import os
import click
from typing import List

from langchain_community.document_loaders import TextLoader, PDFMinerLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
# from langchain.vectorstores import Milvus
from langchain.docstore.document import Document
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INDEXING OF THE DATA


In [ ]:
class LingoChatbotCreateDB:
    def __init__(self):
        # self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.device = 'cuda'
        print(f"Using device: {self.device}")
        self.SOURCE_DIRECTORY = "/content/drive/MyDrive/CHATBOX"
        self.CHROMA_PATH = "/content/CHROMA"
        self.collection_ = "SOCIAL"
        self.embeddings = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base",
                                                model_kwargs={"device": self.device})

        self.vector_store = Chroma(persist_directory=self.CHROMA_PATH,
                                   embedding_function=self.embeddings, collection_name=self.collection_)
        print("Loaded Embedding model!!")

    def load_single_document(self, file_path: str) -> Document:
        # Loads a single document from a file path
        if file_path.endswith(".txt"):
            loader = TextLoader(file_path)
        elif file_path.endswith(".pdf"):
            loader = PDFMinerLoader(file_path)
        elif file_path.endswith(".csv"):
            loader = CSVLoader(file_path)
        return loader.load()[0]

    def load_documents(self, source_dir: str) -> List[Document]:
        # Loads all documents from source documents directory
        all_files = os.listdir(source_dir)
        return [self.load_single_document(f"{source_dir}/{file_path}") for file_path in all_files if
                file_path[-4:] in ['.txt', '.pdf', '.csv']]

    def createDB(self):
        print(f"Loading documents from {self.SOURCE_DIRECTORY}")
        documents = self.load_documents(self.SOURCE_DIRECTORY)
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=150)
        text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=30, separator=" ")
        # text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        print(f"Loaded {len(documents)} documents from {self.SOURCE_DIRECTORY}")
        print(f"Split into {len(texts)} chunks of text")


        print("Indexing the Documents!!!")
        self.vector_store.add_documents(documents=texts)
        print("="*50)
        print("Indexing of the Document Completed")
        print("="*50)

if __name__ == "__main__":
    create_db = LingoChatbotCreateDB()
    create_db.createDB()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-4-123749799.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(persist_directory=self.CHROMA_PATH,


Loaded Embedding model!!
Loading documents from /content/drive/MyDrive/CHATBOX
Loaded 6 documents from /content/drive/MyDrive/CHATBOX
Split into 377 chunks of text
Indexing the Documents!!!
Indexing of the Document Completed


# **RETRIEVAL**



In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from colorama import Fore
import json
import time
import warnings
import os
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# from openai import OpenAI # Remove this import
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI # Import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from colorama import Fore
import time
import warnings
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

warnings.filterwarnings("ignore")


class SLAiAssist:
    def __init__(self):
        t1 = time.time()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(Fore.LIGHTGREEN_EX + "=" * 50, "Initializing Chatbot API", "=" * 50)
        self.CHROMA_PATH = "/content/CHROMA"
        self.collection_ = "SOCIAL"

        # self.model_id = "Qwen/Qwen3-4B" # bigger model (doesn't work on Colab)
        self.model_id = "Qwen/Qwen3-0.6B" #smaller model (works on Colab)
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_id
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id
        )
        self.pipe = pipeline(
            task="text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0,
            repetition_penalty=1.1,
            max_new_tokens=4096,
            return_full_text=True
        )
        self.sl_llm = HuggingFacePipeline(pipeline=self.pipe)

        self.embeddings = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base",
                                                model_kwargs={"device": self.device})

        self.db = Chroma(
            persist_directory=self.CHROMA_PATH,
            embedding_function=self.embeddings,
            collection_name=self.collection_
        )


        self.retriever = self.db.as_retriever()

        self.SLqa = RetrievalQA.from_chain_type(llm=self.sl_llm, chain_type="stuff", retriever=self.retriever,
                                                return_source_documents=True)
        t2 = time.time()
        print(f">> Initializing Time: ", round((t2 - t1), 3))
        print(f"Running on: {self.device}")
        print(Fore.LIGHTCYAN_EX + "|| Loaded Chatbot ||")

    def assist(self, query):
        t1 = time.time()
        print("Chatbot answering the question...")
        res = self.SLqa.invoke({"query": query}) # Use invoke instead of direct call
        t2 = time.time()
        print(res)
        print("Answer Time: ", round((t2 - t1), 3))
        answer = res['result']
        output = {"question": query, "answer": answer}
        print("Chatbot OUTPUT: ", output)
        print("-"*50)
        print("User Query: ", output['question'])
        print("Output: ", output['answer'])

        return output


In [ ]:
Agenassist = SLAiAssist()

================================================== Initializing Chatbot API ==================================================


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


>> Initializing Time:  32.662
Running on: cuda
|| Loaded Chatbot ||


In [ ]:
# query = Ex. "Describe the sixities in context of US history to me."
query = "What was the era of Reconstruction?"
Agenassist.assist(query)

Chatbot answering the question...
{'query': 'What was the era of Reconstruction?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nchange. \n\nunderstanding by discussing the Cold War as a \ncause of the Second Red Scare, 1950s \nconformity, the anti-war movement of the 1960s, \nor the distrust of government in the 1970s. \n\n• “America practiced a policy of containment \n\nduring the Cold War. . . . When America sent \nthousands of drafted young soldiers to Vietnam \nto die, it caused a public upset. The SDS is one \ngroup that protest. College student and young \nbecause politically and socially engaged which \nled to 18 year olds getting the right to vote.” \n\n• Many responses had a chronological \n\nmisconception regarding developments from \nprior periods, discussing them as if they occurred \nduring the period from 1945 to 1980. \n\n• Better responses 

{'question': 'What was the era of Reconstruction?',
 'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nchange. \n\nunderstanding by discussing the Cold War as a \ncause of the Second Red Scare, 1950s \nconformity, the anti-war movement of the 1960s, \nor the distrust of government in the 1970s. \n\n• “America practiced a policy of containment \n\nduring the Cold War. . . . When America sent \nthousands of drafted young soldiers to Vietnam \nto die, it caused a public upset. The SDS is one \ngroup that protest. College student and young \nbecause politically and socially engaged which \nled to 18 year olds getting the right to vote.” \n\n• Many responses had a chronological \n\nmisconception regarding developments from \nprior periods, discussing them as if they occurred \nduring the period from 1945 to 1980. \n\n• Better responses used developments from earlier

# RETRIEVAL THROUGH CUSTOM PROMPT (OUT OF CONTEXT HANDLING MECHANISM)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from colorama import Fore
import time
import warnings
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate

warnings.filterwarnings("ignore")


class SLAiAssist:
    def __init__(self):
        t1 = time.time()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(Fore.LIGHTGREEN_EX + "=" * 50, "Initializing Chatbot API", "=" * 50)
        self.CHROMA_PATH = "/content/CHROMA"
        self.collection_ = "SOCIAL"

        # === Qwen-4B LLM setup ===
        self.model_id = "Qwen/Qwen3-0.6B"
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_id
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id
        )
        self.pipe = pipeline(
            task="text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0,
            repetition_penalty=1.1,
            max_new_tokens=4096,
            return_full_text=True
        )
        self.sl_llm = HuggingFacePipeline(pipeline=self.pipe)

        # Retrieval prompt
        self.template = '''Use the following context to answer the users question. If user question found in the context, return the exact answer from the context and never provide correct answer from your knowledge. Give answer only from the context, never give answer from your knowledge even if you know the answer for out of context question and you must say only "I don't know" in the answer.

                        Context: {context}
                        Question: {question}

                        If answer not found in the context, only say "I don't know" and never provide explanation even though you know the explanation for the given out of context question.
                        Answer:'''
        self.PROMPT = PromptTemplate(
            template=self.template, input_variables=["context", "question"]
        )

        # Embeddings + Chroma
        self.embeddings = HuggingFaceEmbeddings(
            model_name="moka-ai/m3e-base",
            model_kwargs={"device": self.device}
        )
        self.db = Chroma(
            persist_directory=self.CHROMA_PATH,
            embedding_function=self.embeddings,
            collection_name=self.collection_
        )
        self.retriever = self.db.as_retriever()

        # QA chain
        self.SLqa = RetrievalQA.from_chain_type(
            llm=self.sl_llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.PROMPT}
        )

        t2 = time.time()
        print(f">> Initializing Time: {round((t2 - t1), 3)} seconds")
        print(f"Running on: {self.device}")
        print(Fore.LIGHTCYAN_EX + "|| Loaded Chatbot ||")

    def assist(self, query: str) -> dict:
        t1 = time.time()
        print("Chatbot answering the question...")
        res = self.SLqa.invoke({"query": query})
        t2 = time.time()
        print(res)
        print("Answer Time: ", round((t2 - t1), 3), "seconds")
        answer = res['result']
        output = {"question": query, "answer": answer}
        print("Chatbot OUTPUT: ", output)
        print("-"*50)
        print("User Query: ", output['question'])
        print("Output: ", output['answer'])
        return output



In [ ]:
agent = SLAiAssist()

================================================== Initializing Lingo Chatbot API with Qwen-4B ==================================================


Device set to use cuda:0


>> Initializing Time: 9.667 seconds
Running on: cuda
|| Loaded Lingo Chatbot (Qwen-4B) ||


In [ ]:
# Example
query = "Why is the sky blue?"
agent.assist(query)

Lingo Chatbot answering the question...
{'query': 'Why are my orders getting rejected?', 'result': 'Use the following context to answer the users question. If user question found in the context, return the exact answer from the context and never provide correct answer from your knowledge. Give answer only from the context, never give answer from your knowledge even if you know the answer for out of context question and you must say only "I don\'t know" in the answer.\n\n                        Context: please call us on 1800-2100-818 or write us back and we \nwould be glad to assist you. \n\nAssuring you of our best services. Because your Smile is our Reward! \n\nThanks, \nTeam Mirae Asset \n\n19.Order related – Why my orders are getting rejected \n\nHello, \n\nGreetings from m.Stock by Mirae Asset! \n\nWe are writing to you with reference to your query. \n\nKindly follow the below path to view the order rejection message at your end. \n\n• Click on orders \n• Executed order \n• Select

{'question': 'Why are my orders getting rejected?',
 'answer': 'Use the following context to answer the users question. If user question found in the context, return the exact answer from the context and never provide correct answer from your knowledge. Give answer only from the context, never give answer from your knowledge even if you know the answer for out of context question and you must say only "I don\'t know" in the answer.\n\n                        Context: please call us on 1800-2100-818 or write us back and we \nwould be glad to assist you. \n\nAssuring you of our best services. Because your Smile is our Reward! \n\nThanks, \nTeam Mirae Asset \n\n19.Order related – Why my orders are getting rejected \n\nHello, \n\nGreetings from m.Stock by Mirae Asset! \n\nWe are writing to you with reference to your query. \n\nKindly follow the below path to view the order rejection message at your end. \n\n• Click on orders \n• Executed order \n• Select the respective order \n• Select Ord

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.0 MB/s eta 0:00:00
